## Adaboost algorithm implement

Try to implement Adaboost with hands-on tutorial from frist beginning. **Adaboost** is an ensemble algorithm that will use a ensemble of stamp to make prediction and training, the main step for Adaboost is followed as bellow:

1. Give each sample with a weight, first time should be: 1/n_sample.
2. Build a stump from features that we have based on Gini or Entropy.
3. Evaluate model for each class with correct prediction and wrong. 
4. Compute `amount for say` with 1/2 * log((1 - total_error)/total_error).
5. Update each sample with new weights, weight is computed by formula:
   For correct prediction sample: 
                                weight * e(-amount_of_say)
   For in-correct prediction sample:
                                weight * e(amount_of_say)
   Tips:
   There is a main idea to update each sample weight based on `how well the stamp does`. For example, for in-correct sample weight update, if current stamp does well, then `amount_of_say` will be reall large, then in-correct weight will be greater than before, if current stamp doesn't do well, then `amount_of_say` will be smaller, so in theory if current stamp doesn't do well for in-correct sample, then current  weights will be bigger, as for correct prediction sample will be smaller.
6. Create a new stamp based on weighted Gini Index or we could create a new sample datasets based on the weights with repeated selection of samples for larger weights.
7. Step repeated.


Start to implement with **Adaboost**.


In [80]:
import numpy as np
from collections import Counter

# random data and label for easy doing
data = np.array([[3, 3, 6, 4, 6, 3], [1.2, 2.1, 2.3, 3.4, 5.3, 1.8]]).T
label = np.array([1, 0, 1, 1, 1, 0])

weight = np.array([1/len(label) for _ in range(len(label))])

In [81]:
# First should with Gini index compute
def gini_index(y):
    """compute each class times with 1 - sum(prob**2)"""
    counter = list(Counter(y).values())
    probs = [d / len(y) for d in counter]

    gini = 1 - sum([prob **2 for prob in probs])

    return gini

print("Root gini: ", gini_index(label))

Root gini:  0.4444444444444444


### Stamp creation

Next step should create a stamp based on each feature with best gini index as a root stamp.

Category columns to process.

In [82]:
# First try 1th column, get unique data and to get best split gini
first_data = data[:, 0]
unique_values = np.unique(first_data)

root_gini = gini_index(label)

gini_list = []
# loop each columns with equal to that value to get label.
for val in unique_values:
    # so we could create a binary stamp.
    val_left_label = label[first_data == val]
    val_right_label = label[first_data != val]

    val_left_gini = gini_index(val_left_label)
    val_right_gini = gini_index(val_right_label)

    val_gini = val_left_gini + val_right_gini - root_gini

    gini_list.append(val_gini)

# select max gini-index value's index to select root step value
max_gini_cat = max(gini_list)
select_value_cat = unique_values[np.argmax(gini_list)]
print("Selected value: {} with max gini: {} to split.".format(select_value_cat, max_gini_cat))

Selected value: 6.0 with max gini: 0.05555555555555558 to split.


Continues data to process

In [83]:
second_data = data[:, 1]

# sort them and get median value
second_data = sorted(second_data)
median_data = [(second_data[i] + second_data[i+1])/2 for i in range(len(second_data) - 1)]

root_gini = gini_index(label)

gini_list = []

# loop each data with lower and higher to make a binary tree
for val in median_data:
    val_left_label = label[second_data <=  val]
    val_right_label = label[second_data >  val]

    val_left_gini = gini_index(val_left_label)
    val_right_gini = gini_index(val_right_label)

    val_gini = val_left_gini + val_right_gini - root_gini

    gini_list.append(val_gini)


# select max gini-index value's index to select root step value
max_gini_con = max(gini_list)
select_value_con = median_data[np.argmax(gini_list)]
print("Selected value: {} with max gini: {} to split.".format(select_value_con, max_gini_con))

Selected value: 2.2 with max gini: 0.4444444444444444 to split.


Select max gini value as first stamp. So we could get that for now, to select 2th column with median value: 1.95 will get max gini index. So let's select it as root stamp value.

In [102]:
select_col = 1
select_val = select_value_con

# we need to make prediction! so the prediction is based on how many label in that leaves
left_label = label[data[:, select_col] <= select_val]
right_label = label[data[:, select_col] > select_val]

left_weight = weight[data[:, select_col] <= select_val]
right_weight = weight[data[:, select_col] > select_val]

print(left_label, right_label)

# the prediction is the max number of that leaves. so for now is both of them are predict as 1.
left_pred = Counter(left_label).most_common()[0][0]
right_pred = Counter(right_label).most_common()[0][0]

print("Left pred: {}, right pred: {}".format(left_pred, right_pred))


# make this stamp into a function:
def stamp_pred(data, select_col=1):
    select_val = select_value_con
    
    if data[select_col] <= select_val:
        return 0
    else:
        return 1

[1 0 0] [1 1 1]
Left pred: 0, right pred: 1


Compute amount of say with: 1/2*log((1-total_error)/total_error)

In [99]:
# left not equal to 0 should be mis-correct, right not equal to 1 is mis-correct
left_error = [x != 0 for x in left_label]
right_error = [x != 1 for x in right_label]

# total_error should based on the weight, not with number
total_error = sum(left_weight[left_error].tolist() + right_weight[right_error].tolist())

amount_of_say = 1/2 * np.log((1 - total_error) / total_error)

print("This stamp amount of say is : {}".format(amount_of_say))

This stamp amount of say is : 0.8047189562170503


In [103]:
# loop each data point to get prediction 
pred = np.array([stamp_pred(x) for x in data])


def weight_update(pred, weight):
    # based on correct pred or not.
    
    # This is used to get different type for correct and not weight update policy.
    correct_pred = label == pred
    new_weight = np.empty_like(weight)

    for i in range(len(weight)):
        if correct_pred[i]:
            # correct:
            new_w = weight[i] * np.exp(-amount_of_say)
        else:
            new_w = weight[i] * np.exp(amount_of_say)
        new_weight[i] = new_w

    # Normalize new_weight 
    new_weight /= np.sum(new_weight)

    return new_weight, correct_pred

new_weight, correct_or_not = weight_update(pred, weight)

# So that we do see that for correct pred's weight is smaller and miss-correct's weight is larger.
print(correct_or_not)
print(new_weight)


[False  True  True  True  True  True]
[0.5 0.1 0.1 0.1 0.1 0.1]


So we could try to build a new dataset based on these weights. Let's try to make a random value and get the first satisfied value index, and add it into new dataset.

In [106]:
cumsum = np.cumsum(new_weight)

new_dataset = []

for i in range(data.shape[0]):
    r = np.random.random()
    select_index = (r < cumsum).tolist().index(True)

    new_dataset.append(data[select_index])

new_dataset = np.array(new_dataset)

print("Get new dataset")
print(new_dataset)

Get new dataset
[[3.  1.2]
 [3.  1.2]
 [3.  1.2]
 [3.  1.2]
 [3.  1.2]
 [3.  2.1]]


Then we could repeat these steps to build another stamp, but one more thing is that each tree will have a attribute with **amount_of_say** to demonstrate importance of each tree. To make prediction, like we have 10 stamps, then we will get different pred, we could use **voting** to get prediction.